In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize as wt 
from nltk.corpus import stopwords

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
from nltk.stem import wordnet
from nltk.stem import WordNetLemmatizer
import spacy
nlp = spacy.load("en_core_web_sm")
lemmatizer = WordNetLemmatizer()

In [13]:
df_qs= pd.read_csv('questions.csv',header=0)
df_qs.head(2)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0


In [14]:
new_qs_df = df_qs.sample(40000)
new_qs_df.drop(['id'],axis=1,inplace=True)

In [7]:
def remove_stopwords(text):
    stopWords = set(stopwords.words('english'))
    words = wt(text)
    filtered_words = [word for word in words if word.lower() not in stopWords]
    return ' '.join(filtered_words)

In [15]:
new_qs_df['question1']=new_qs_df['question1'].apply(lambda x: str(x).replace('.', '').replace(',', '').replace("'", '').replace('"', '').replace('-',' ').replace('&','and').replace('?',''))
new_qs_df['question2']=new_qs_df['question2'].apply(lambda x: str(x).replace('.', '').replace(',', '').replace("'", '').replace('"', '').replace('-',' ').replace('&','and').replace('?',''))

In [16]:
new_qs_df['question1']=new_qs_df['question1'].apply(lambda x:remove_stopwords(x))
new_qs_df['question2']=new_qs_df['question2'].apply(lambda x:remove_stopwords(x))

In [17]:
new_qs_df

,qid1,qid2,question1,question2,is_duplicate
12545,25047,25048,age appropriate wear bold lipstick,lipstick wear red saree,0
122332,153775,242414,cheapest place live San Francisco bootstrappin...,new grads afford living somewhere like NYC San...,0
166595,329273,329274,museum,museum,0
343122,672512,672513,talk English fluently,speak English fluency,1
274930,540604,540605,colleges look cumulative gpa four years high s...,356 GPA good sophomore year high school,0
...,...,...,...,...,...
152007,300683,300684,government India reducing price fuel crude oil...,government India reducing price fuel crude oil...,0
295356,580191,580192,people dont judge based gender would people st...,gender largely social construct would someone ...,0
360696,706235,706236,block Pingdom Google Chrome,way block ads Google chrome mobile application,0
399279,780082,780083,braingasmic non fiction reads,must read non fiction books,0


In [22]:
new_qs_df['question1'][152007]

'government India reducing price fuel crude oil lowest 2016'

In [24]:
def total_characters(text):
    return len(text)
def total_words(text):
    return len(text.split())
print(total_characters(new_qs_df['question1'][152007]),' ',total_characters(new_qs_df['question2'][152007]))
print(total_words(new_qs_df['question1'][152007]),' ',total_words(new_qs_df['question2'][152007]))


58   58
9   9


In [25]:
new_qs_df['total_characters_q1']=new_qs_df['question1'].apply(lambda x:total_characters(x))
new_qs_df['total_characters_q2']=new_qs_df['question2'].apply(lambda x:total_characters(x))
new_qs_df['total_words_q1']=new_qs_df['question1'].apply(lambda x:total_words(x))
new_qs_df['total_words_q2']=new_qs_df['question2'].apply(lambda x:total_words(x))

In [26]:
new_qs_df

,qid1,qid2,question1,question2,is_duplicate,total_characters_q1,total_characters_q2,total_words_q1,total_words_q2
12545,25047,25048,age appropriate wear bold lipstick,lipstick wear red saree,0,34,23,5,4
122332,153775,242414,cheapest place live San Francisco bootstrappin...,new grads afford living somewhere like NYC San...,0,55,73,7,12
166595,329273,329274,museum,museum,0,6,6,1,1
343122,672512,672513,talk English fluently,speak English fluency,1,21,21,3,3
274930,540604,540605,colleges look cumulative gpa four years high s...,356 GPA good sophomore year high school,0,51,39,8,7
...,...,...,...,...,...,...,...,...,...
152007,300683,300684,government India reducing price fuel crude oil...,government India reducing price fuel crude oil...,0,58,58,9,9
295356,580191,580192,people dont judge based gender would people st...,gender largely social construct would someone ...,0,86,66,15,8
360696,706235,706236,block Pingdom Google Chrome,way block ads Google chrome mobile application,0,27,46,4,7
399279,780082,780083,braingasmic non fiction reads,must read non fiction books,0,29,27,4,5


In [33]:
def lemma(text):
    lemm = [word.lemma_ for word in nlp(text)]
    return ' '.join(lemm)


In [34]:
new_qs_df['question1']=new_qs_df['question1'].apply(lambda x:lemma(x))
new_qs_df['question2']=new_qs_df['question2'].apply(lambda x:lemma(x))

In [ ]:
new_qs_df['question1']=new_qs_df['question1'].apply(lambda x: wt(x))
new_qs_df['question2']=new_qs_df['question2'].apply(lambda x: wt(x))

In [38]:
ques=list(new_qs_df['question1'])+list(new_qs_df['question2'])

In [39]:
import numpy as np

In [42]:
cv = CountVectorizer(max_features=3000)
q1_ar,q2_ar = np.vsplit(cv.fit_transform(ques).toarray(),2)


In [44]:
tdf1=pd.DataFrame(q1_ar,index=new_qs_df.index)
tdf2=pd.DataFrame(q2_ar,index=new_qs_df.index)
tdf = pd.concat([tdf1,tdf2],axis=1)

In [46]:
final_df = new_qs_df.drop(columns=['qid1','qid2','question1','question2'])

In [45]:
tdf.shape

(40000, 6000)

In [47]:
final_df=pd.concat([final_df, tdf], axis=1)
final_df.shape

(40000, 6005)

In [49]:
final_df.sample(5)

,is_duplicate,total_characters_q1,total_characters_q2,total_words_q1,total_words_q2,0,1,2,3,4,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
143169,0,28,35,5,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375405,1,12,38,2,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150837,1,26,40,3,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
212537,1,33,26,4,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
177302,0,44,46,5,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0].values,test_size=0.3,random_state=1)

In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7474166666666666

In [51]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.729